In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

In [ ]:
players <- read_csv("players.csv")
players

players_select <- players |>
    filter(played_hours != 0.0) |>
    select(experience, played_hours, Age)
players_select

players_split <- initial_split(players_select, prop = 0.75, strata = played_hours)
players_training <- training(players_split)
players_training
players_testing <- testing(players_split)
players_testing

In [ ]:
players_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
       set_engine("kknn") |>
       set_mode("regression") 

players_recipe <- recipe(played_hours ~ ., data = players_training) |>
       step_scale(all_predictors()) |>
       step_center(all_predictors())

players_vfold <- vfold_cv(players_training, v = 5, strata = played_hours)

players_workflow <- workflow() |> 
    add_recipe(players_recipe) |> 
    add_model(players_spec)
players_workflow

In [ ]:
gridvals <- tibble(neighbors = seq(from = 1, to = 50, by = 10))

players_results <- workflow() |> 
    add_recipe(players_recipe) |> 
    add_model(players_spec) |>
    tune_grid(object = players_workflow, resamples = players_vfold, grid = gridvals) |>
    collect_metrics()
players_results